In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import time
import os
import json

class NSEStockIndexAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_index_analysis'
        self.results_dir = os.path.join(self.base_dir, 'index_results')
        os.makedirs(self.results_dir, exist_ok=True)
        
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.get("http://nseindia.com", headers=self.headers)
        
        # List of indices to analyze
        self.equity_market_categories = [
            'NIFTY 50', 'NIFTY ENERGY', 'NIFTY MIDCAP 50', 'NIFTY SMALLCAP 50', 'NIFTY AUTO', 'NIFTY BANK', 'NIFTY FINANCIAL SERVICES', 'NIFTY FMCG', 'NIFTY IT', 'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY REALTY', 'NIFTY PRIVATE BANK', 'NIFTY INFRASTRUCTURE', 'NIFTY MNC', 'NIFTY PSE'," NIFTY CONSUMER DURABLES"]

    def equity_market_data(self, category, symbol_list=False):
        """
        Retrieve stock data for a specific index using NSE API
        
        Args:
            category (str): Index name
            symbol_list (bool): If True, return only list of symbols
        
        Returns:
            list or DataFrame: Stocks in the specified index
        """
        try:
            category = category.upper().replace(' ', '%20').replace('&', '%26')
            response = self.session.get(
                f"https://www.nseindia.com/api/equity-stockIndices?index={category}", 
                headers=self.headers
            )
            
            if response.status_code != 200:
                print(f"Error fetching data for {category}: HTTP {response.status_code}")
                return []
            
            data = response.json()["data"]
            df = pd.DataFrame(data)
            df = df.drop(["meta"], axis=1)
            df = df.set_index("symbol", drop=True)
            
            if symbol_list:
                return list(df.index)
            else:
                return df
        
        except Exception as e:
            print(f"Error retrieving stocks for {category}: {e}")
            return []

    def calculate_rsi(self, prices_tuple, periods=14):
        """RSI calculation method"""
        try:
            prices = pd.Series(prices_tuple)
            
            if len(prices) < periods + 1:
                return None
                
            # Calculate price changes
            delta = prices.diff()
            
            # Split gains and losses
            gains = delta.copy()
            losses = delta.copy()
            
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calculate initial averages
            first_avg_gain = gains[:periods+1].mean()
            first_avg_loss = losses[:periods+1].mean()
            
            avg_gains = [first_avg_gain]
            avg_losses = [first_avg_loss]
            
            # Calculate subsequent values
            for i in range(periods+1, len(gains)):
                avg_gain = (avg_gains[-1] * (periods-1) + gains[i]) / periods
                avg_loss = (avg_losses[-1] * (periods-1) + losses[i]) / periods
                avg_gains.append(avg_gain)
                avg_losses.append(avg_loss)
            
            # Calculate final RSI
            if avg_losses[-1] == 0:
                rsi = 100.0
            else:
                rs = avg_gains[-1] / avg_losses[-1]
                rsi = 100 - (100 / (1 + rs))
            
            return float(rsi)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def get_stock_data(self, symbol):
        """Fetch detailed stock data"""
        try:
            time.sleep(1)  # Rate limiting
            ticker = yf.Ticker(f"{symbol}.NS")
            
            # Get stock info
            info = ticker.info
            
            # Get historical data
            hist_data = ticker.history(period="1y", interval="1d")

            if hist_data.empty:
                return None

            # Calculate basic metrics
            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            
            latest_rsi = self.calculate_rsi(prices_tuple)
            
            # Prepare stock information dictionary
            stock_info = {
                'Symbol': symbol,
                'Latest_Price': round(latest_price, 2),
                'RSI': round(latest_rsi, 2) if latest_rsi is not None else 'N/A',
                'Market_Cap': info.get('marketCap', 'N/A'),
                'Sector': info.get('sector', 'N/A'),
                'Industry': info.get('industry', 'N/A'),
                'PE_Ratio': round(info.get('trailingPE', 'N/A'), 2) if isinstance(info.get('trailingPE'), (int, float)) else 'N/A',
                '52W_High': round(info.get('fiftyTwoWeekHigh', 'N/A'), 2) if isinstance(info.get('fiftyTwoWeekHigh'), (int, float)) else 'N/A',
                '52W_Low': round(info.get('fiftyTwoWeekLow', 'N/A'), 2) if isinstance(info.get('fiftyTwoWeekLow'), (int, float)) else 'N/A',
                'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 'N/A'
            }
            
            return stock_info

        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            return None

    def analyze_indices_stocks(self):
        """Analyze stocks for each index and generate Excel report"""
        # Timestamp for unique filename
        current_date = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_file = os.path.join(self.results_dir, f'index_stocks_analysis_{current_date}.xlsx')
        
        # Prepare Excel writer
        with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
            for index_category in self.equity_market_categories:
                try:
                    # Get stocks for this index
                    stocks = self.equity_market_data(index_category, symbol_list=True)
                    
                    # Process stocks for this index
                    results = []
                    for symbol in stocks:
                        stock_data = self.get_stock_data(symbol)
                        if stock_data:
                            results.append(stock_data)
                    
                    # Convert to DataFrame
                    if results:
                        df = pd.DataFrame(results)
                        
                        # Write to Excel sheet named after the index
                        sheet_name = index_category[:31]  # Excel sheet names limited to 31 chars
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
                        print(f"Processed {len(results)} stocks for {index_category}")
                
                except Exception as e:
                    print(f"Error processing {index_category}: {e}")
        
        print(f"\nAnalysis complete. Results saved to {excel_file}")

if __name__ == "__main__":
    analyzer = NSEStockIndexAnalyzer()
    analyzer.analyze_indices_stocks()

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import time
import os
import json

class NSEStockIndexAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_index_analysis'
        self.results_dir = os.path.join(self.base_dir, 'index_results')
        os.makedirs(self.results_dir, exist_ok=True)
        
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.get("http://nseindia.com", headers=self.headers)
        
        # List of indices to analyze
        self.equity_market_categories = [
            'NIFTY 50', 'NIFTY ENERGY', 'NIFTY MIDCAP 50', 'NIFTY SMALLCAP 50', 'NIFTY AUTO', 'NIFTY BANK', 'NIFTY FINANCIAL SERVICES', 'NIFTY FMCG', 'NIFTY IT', 'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY REALTY', 'NIFTY PRIVATE BANK', 'NIFTY INFRASTRUCTURE', 'NIFTY MNC', 'NIFTY PSE', 'NIFTY CONSUMER DURABLES']

    def equity_market_data(self, category, symbol_list=False):
        """
        Retrieve stock data for a specific index using NSE API
        
        Args:
            category (str): Index name
            symbol_list (bool): If True, return only list of symbols
        
        Returns:
            list or DataFrame: Stocks in the specified index
        """
        try:
            category = category.upper().replace(' ', '%20').replace('&', '%26')
            response = self.session.get(
                f"https://www.nseindia.com/api/equity-stockIndices?index={category}", 
                headers=self.headers
            )
            
            if response.status_code != 200:
                print(f"Error fetching data for {category}: HTTP {response.status_code}")
                return []
            
            data = response.json()["data"]
            df = pd.DataFrame(data)
            df = df.drop(["meta"], axis=1)
            df = df.drop(df.index[0])
            df = df.set_index("symbol", drop=True)
            
            if symbol_list:
                return list(df.index)
            else:
                return df
        
        except Exception as e:
            print(f"Error retrieving stocks for {category}: {e}")
            return []

    def calculate_rsi(self, prices_tuple, periods=14):
        """RSI calculation method"""
        try:
            prices = pd.Series(prices_tuple)
            
            if len(prices) < periods + 1:
                return None
                
            # Calculate price changes
            delta = prices.diff()
            
            # Split gains and losses
            gains = delta.copy()
            losses = delta.copy()
            
            gains[gains < 0] = 0
            losses[losses > 0] = 0
            losses = abs(losses)
            
            # Calculate initial averages
            first_avg_gain = gains[:periods+1].mean()
            first_avg_loss = losses[:periods+1].mean()
            
            avg_gains = [first_avg_gain]
            avg_losses = [first_avg_loss]
            
            # Calculate subsequent values
            for i in range(periods+1, len(gains)):
                avg_gain = (avg_gains[-1] * (periods-1) + gains[i]) / periods
                avg_loss = (avg_losses[-1] * (periods-1) + losses[i]) / periods
                avg_gains.append(avg_gain)
                avg_losses.append(avg_loss)
            
            # Calculate final RSI
            if avg_losses[-1] == 0:
                rsi = 100.0
            else:
                rs = avg_gains[-1] / avg_losses[-1]
                rsi = 100 - (100 / (1 + rs))
            
            return float(rsi)
            
        except Exception as e:
            print(f"Error calculating RSI: {e}")
            return None

    def calculate_tema(self, prices, period=50):
        """
        Calculate Triple Exponential Moving Average (TEMA)
        
        Args:
            prices (pd.Series): Price series
            period (int): Period for TEMA calculation
        
        Returns:
            float: TEMA value
        """
        try:
            if len(prices) < period:
                return None
            
            # Calculate EMAs
            ema1 = prices.ewm(span=period, adjust=False).mean()
            ema2 = ema1.ewm(span=period, adjust=False).mean()
            ema3 = ema2.ewm(span=period, adjust=False).mean()
            
            # TEMA Formula: TEMA = 3 * EMA1 - 3 * EMA2 + EMA3
            tema = 3 * ema1.iloc[-1] - 3 * ema2.iloc[-1] + ema3.iloc[-1]
            
            return float(tema)
        
        except Exception as e:
            print(f"Error calculating TEMA: {e}")
            return None

    def get_stock_data(self, symbol):
        """Fetch detailed stock data"""
        try:
            time.sleep(1)  # Rate limiting
            ticker = yf.Ticker(f"{symbol}.NS")
            
            # Get stock info
            info = ticker.info
            
            # Get historical data for 1 year
            hist_data = ticker.history(period="1y", interval="1d")

            # Check if 1 year data is available, else get maximum available data
            if hist_data.empty:
                hist_data = ticker.history(period="max", interval="1d")
                if hist_data.empty:
                    return None

            # Calculate basic metrics
            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            
            # Calculate indicators
            latest_rsi = self.calculate_rsi(prices_tuple)
            
            # Calculate EMA-50 and TEMA-50
            prices_series = hist_data['Close']
            ema_50 = prices_series.ewm(span=50, adjust=False).mean().iloc[-1]
            tema_50 = self.calculate_tema(prices_series)
            
            # Prepare stock information dictionary
            stock_info = {
                'Symbol': symbol,
                'Latest_Price': round(latest_price, 2),
                'RSI': round(latest_rsi, 2) if latest_rsi is not None else 'N/A',
                'EMA_50': round(ema_50, 2) if ema_50 is not None else 'N/A',
                'TEMA_50': round(tema_50, 2) if tema_50 is not None else 'N/A',
                'TEMA_50-EMA_50': round(tema_50 - ema_50, 2) if tema_50 is not None and ema_50 is not None else 'N/A',
                'Market_Cap': info.get('marketCap', 'N/A'),
                'Sector': info.get('sector', 'N/A'),
                'Industry': info.get('industry', 'N/A'),
                'PE_Ratio': round(info.get('trailingPE', 'N/A'), 2) if isinstance(info.get('trailingPE'), (int, float)) else 'N/A',
                '52W_High': round(info.get('fiftyTwoWeekHigh', 'N/A'), 2) if isinstance(info.get('fiftyTwoWeekHigh'), (int, float)) else 'N/A',
                '52W_Low': round(info.get('fiftyTwoWeekLow', 'N/A'), 2) if isinstance(info.get('fiftyTwoWeekLow'), (int, float)) else 'N/A',
                'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 'N/A'
            }
            
            return stock_info

        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            return None

    def analyze_indices_stocks(self):
        """Analyze stocks for each index and generate Excel report"""
        # Timestamp for unique filename
        current_date = datetime.now().strftime('%Y%m%d_%H%M%S')
        excel_file = os.path.join(self.results_dir, f'index_stocks_analysis_{current_date}.xlsx')
        
        # Prepare Excel writer
        with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
            for index_category in self.equity_market_categories:
                try:
                    # Get stocks for this index
                    stocks = self.equity_market_data(index_category, symbol_list=True)
                    
                    # Process stocks for this index
                    results = []
                    for symbol in stocks:
                        stock_data = self.get_stock_data(symbol)
                        if stock_data:
                            results.append(stock_data)
                    
                    # Convert to DataFrame
                    if results:
                        df = pd.DataFrame(results)
                        
                        # Write to Excel sheet named after the index
                        sheet_name = index_category[:31]  # Excel sheet names limited to 31 chars
                        df.to_excel(writer, sheet_name=sheet_name, index=False)
                        
                        print(f"Processed {len(results)} stocks for {index_category}")
                
                except Exception as e:
                    print(f"Error processing {index_category}: {e}")
        
        print(f"\nAnalysis complete. Results saved to {excel_file}")

if __name__ == "__main__":
    analyzer = NSEStockIndexAnalyzer()
    analyzer.analyze_indices_stocks()

Processed 50 stocks for NIFTY 50
Processed 40 stocks for NIFTY ENERGY
Processed 50 stocks for NIFTY MIDCAP 50
Processed 50 stocks for NIFTY SMALLCAP 50
Processed 15 stocks for NIFTY AUTO
Processed 12 stocks for NIFTY BANK
Processed 20 stocks for NIFTY FINANCIAL SERVICES
Processed 15 stocks for NIFTY FMCG
Processed 10 stocks for NIFTY IT
Processed 10 stocks for NIFTY MEDIA
Processed 15 stocks for NIFTY METAL
Processed 20 stocks for NIFTY PHARMA
Processed 12 stocks for NIFTY PSU BANK
Processed 10 stocks for NIFTY REALTY
Processed 10 stocks for NIFTY PRIVATE BANK
Processed 30 stocks for NIFTY INFRASTRUCTURE
Processed 30 stocks for NIFTY MNC
Processed 20 stocks for NIFTY PSE
Processed 15 stocks for NIFTY CONSUMER DURABLES

Analysis complete. Results saved to stock_index_analysis\index_results\index_stocks_analysis_20250127_161014.xlsx


In [38]:
print(NSEStockIndexAnalyzer().equity_market_data('NIFTY 50')[1::])


            priority     identifier      open   dayHigh    dayLow  lastPrice  \
symbol                                                                         
HINDUNILVR         0  HINDUNILVREQN   2332.85   2380.80   2323.00    2380.10   
BRITANNIA          0   BRITANNIAEQN   5037.70   5108.00   5013.15    5100.00   
EICHERMOT          0   EICHERMOTEQN   5134.00   5224.50   5115.05    5190.25   
GRASIM             0      GRASIMEQN   2466.35   2514.05   2453.45    2495.00   
ICICIBANK          0   ICICIBANKEQN   1203.10   1218.00   1202.00    1213.70   
TATACONSUM         0  TATACONSUMEQN    988.75   1004.00    986.40     992.80   
WIPRO              0       WIPROEQN    319.00    322.50    316.60     319.95   
TECHM              0       TECHMEQN   1719.00   1736.40   1705.35    1724.00   
POWERGRID          0   POWERGRIDEQN    295.30    302.50    294.35     296.00   
INFY               0        INFYEQN   1865.50   1894.90   1864.25    1875.00   
NESTLEIND          0   NESTLEINDEQN   22

In [2]:
import requests

def fetch_hdfc_option_chain():
    symbol = "HDFC"
    base_url = "https://www.nseindia.com"
    option_chain_url = f"{base_url}/api/option-chain-equities?symbol={symbol}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
    }

    session = requests.Session()
    session.get(base_url, headers=headers)
    response = session.get(option_chain_url, headers=headers)

    if response.status_code == 200:
        try:
            # Parse JSON response
            data = response.json()
            print("Raw JSON response:\n", data)  # Debugging
            
            # Extracting options if available
            records = data.get("records", {})
            options = records.get("data", [])
            
            if options:
                print(f"Option chain data for {symbol}:\n")
                for entry in options[:5]:  # Limit to 5 entries for brevity
                    ce = entry.get("CE", {})
                    pe = entry.get("PE", {})
                    print(
                        f"Strike Price: {ce.get('strikePrice')} | "
                        f"Call OI: {ce.get('openInterest')} | "
                        f"Put OI: {pe.get('openInterest')}"
                    )
            else:
                print("No options data found.")
        except Exception as e:
            print(f"Error processing data: {e}")
    else:
        print(f"Failed to fetch data. HTTP Status Code: {response.status_code}")


fetch_hdfc_option_chain()


Raw JSON response:
 {}
No options data found.
